In [7]:
# !wget https://huggingface.co/datasets/mesolitica/synthetic-jawi-conversation/resolve/main/synthetic-jawi-conversation.jsonl

In [3]:
from malaysian_sft import accept as accept_fn
import json
import random
from tqdm import tqdm
from glob import glob

In [4]:
system = [
    'you are a chatbot that always reply in jawi',
    'you are an assistant that always reply in jawi',
    'awak adalah chatbot yang sentiasa respond dalam jawi',
    'awak adalah pembantu AI yang reply in jawi',
]

roles = {
    'user': '<manusia>',
    'assistant': '<bot>'
}

In [8]:
instructions, rag = [], []

with open('synthetic-jawi-conversation.jsonl') as fopen:
    for l in tqdm(fopen):
        l = json.loads(l)
        if l[0]['role'] == 'context':
            context = l[0]['content']
            f = rag
            i = 1
        else:
            context = ''
            f = instructions
            i = 0
        
        s = random.choice(system)
        
        a = []
        
        for no, i in enumerate(l[i:]):
            if no == 0:
                if len(context):
                    t = f'{s}\n\n```{context}```'
                else:
                    t = f'{s}\n\n'
            else:
                t = ''
                
            if no % 2 == 0:
                c = random.choice(['content_ms', 'content_jawi'])
            else:
                c = 'content_jawi'
                
            t = t + i[c]
            role = roles[i['role']]
            s = f"{role}: {t}"
            a.append(s)
        
        if len(a) % 2 != 0:
            a = a[:-1]
        
        f.append('\n'.join(a))

509375it [00:33, 15117.83it/s]


In [10]:
instructions[1]

'<manusia>: awak adalah pembantu AI yang reply in jawi\n\nHello chatbot, apakah 2 tambah 2 sama?\n<bot>: هيللو! جواڤنڽ اياله ٤.\n<manusia>: Okay, bagaimana pula dengan 5 tambah 3?\n<bot>: ايت سام دڠن ٨.\n<manusia>: ماري چوبا يڠ لبيه سوكر. اڤ يڠ ١٠٠ دبهاڬيكن دڠن ٠؟\n<bot>: ساي مينتا معاف، تتاڤي ڤمبهاڬين دڠن سيفر تيدق دتنتوكن دالم ماتماتيك.\n<manusia>: Oh, saya terlupa tentang itu. Bagaimana pula dengan 2 hingga kuasa 1000?\n<bot>: ثات ووالد ايقوال ١٠٧١٥٠٨٦٠٧١٨٦٢٦٧٣٢٠٩٤٨٤٢٥٠٤٩٠٦٠٠٠١٨١٠٥٦١٤٠٤٨١١٧٠٥٥٣٣٦٠٧٤٤٣٧٥٠٣٨٨٣١٠٦٦٥٥٥٥٢٣٢٣١٧٠٤٧٢٤٠٠٠٠٠٠٠٠٠٠٠٠٠.\n<manusia>: Wah, baguslah awak! Okay, satu soalan terakhir. Apakah punca kuasa dua bagi-1?\n<bot>: ڤونچا كواس دوا باڬي-١ اياله نومبور خيالن، دواكيلي سباڬاي ءاء دالم ماتماتيك.'

In [11]:
filtered = []
for f in instructions:
    filtered.append({
        'prompt_input': None,
        'input': f,
        'output': None,
    })

In [12]:
import pandas as pd

pd.DataFrame(filtered).to_parquet('prepared-force-jawi.parquet')

In [ ]:
from huggingface_hub import HfApi
api = HfApi()

api.upload_file(
    path_or_fileobj="prepared-force-jawi.parquet",
    path_in_repo='data/force_jawi-00000-of-00001.parquet',
    repo_id="mesolitica/Malaysian-SFT",
    repo_type="dataset",
)

prepared-force-jawi.parquet:   0%|          | 0.00/1.01G [00:00<?, ?B/s]